<a href="https://colab.research.google.com/github/hackathon-mercedes-benz-mad/team_14/blob/master/DatasetsPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento de los datasets

El siguiente conjunto de scripts están pensados para ejecutarse sobre la plataforma Google Collab. 
Sigue los pasos:

1.   Ir a **Archivos** (menú izquierdo).
2.   **Crear carpeta** con el nombre "*raw_datasets*".
3.   **Cargar los datasets dentro de la carpeta** que acabamos de crear (tal y como se nos han entregado, .csv, .json, .xml, etc). 
4.  Los datasets deberían ahora encontrarse en `/raw_datasets/` (la URI para python es `/content/raw_datasets/`).
5.   Comenzar a ejecutar las funciones (top-bottom).

From now on, the rest of the documentation will be in english,



In [88]:
# Libraries
import time
import numpy as np
import pandas as pd
import io
from google.colab import files

In [89]:
### Generic functions
def CheckDatasets(datasets):
  for dataset in datasets:
    try:
      open("/content/raw_datasets/"+dataset, "r")
    except IOError:
      return 0, dataset

  return 1

In [90]:
# Training settings + parameters
__seed = 1

# Random seed
np.random.seed(__seed)

In [91]:
# Load Data
print("Loading datasets... ")

dataset_names = ["Anuncios_usados.csv",
                 "Modelos.json",
                 "Precio_historico.xml",
                 "Tipo_de_cambio.csv",
                 "Ventas_nuevas.json",
                 "Versiones.csv",
                 "inflation_data.csv"]

if not CheckDatasets(dataset_names):
  print("Error, some datasets where not found, check the README or BOF (Beginning of File).")
else:
  print("Datasets are OK!")

Loading datasets... 
Datasets are OK!


# Prunning datasets

Now that the datasets are loaded, we can start removing the columns and cleaning the datasets, once done, we can export the new datasets (to upload them to github) or feed them directly into the prediction model.

In [92]:
# 1011 rows, 3 cols
fields = ['Automaker', 'Genmodel', 'Genmodel_ID']
modelos = pd.read_json("/content/raw_datasets/Modelos.json")

modelos = modelos[fields]
clean_modelos = modelos.dropna()

#clean_modelos.head()

In [93]:
# 773 rows, 21 cols
ventas_nuevas = pd.read_json("/content/raw_datasets/Ventas_nuevas.json")

ventas_nuevas = ventas_nuevas.drop('Genmodel', axis=1)
ventas_nuevas = ventas_nuevas.drop('Maker', axis=1)

clean_ventas_nuevas = ventas_nuevas.dropna()

#clean_ventas_nuevas.head()

In [94]:
# 13330 rows, 2 cols
fields = ['Genmodel_ID', 'Year', 'Price']
versiones = pd.read_csv("/content/raw_datasets/Versiones.csv", skipinitialspace=True, usecols=fields)

# Remove _ from Genmodel_ID
versiones['Genmodel_ID'] = versiones['Genmodel_ID'].map(lambda x: x.replace('_',''))

# New pandas dataframe
fields = ['Genmodel_ID', 'Reg_year', 'AvgPrice']
clean_df = pd.DataFrame(columns=fields)

# Sum of the different prices
t_gmodel = -1
t_year = -1
price_accum = 0
counter = 0
for idx, row in versiones.iterrows():
  if row['Genmodel_ID'] != t_gmodel:
    t_gmodel = row['Genmodel_ID']

    if row['Year'] != t_year:
      t_year = row['Year']
      price_accum = row['Price']
      counter = 1

  else:
    if row['Year'] == t_year:
      counter+=1
      price_accum+=row['Price']
    else:
      price_accum/=counter
      counter = 1
      clean_df = clean_df.append({'Genmodel_ID': t_gmodel, 'Reg_year': t_year, 'AvgPrice': price_accum}, ignore_index=True)
      price_accum = row['Price']
      t_year = row['Year']

#clean_df.head()

In [95]:
# 272 rows, 2 cols
inflation = pd.read_csv("/content/raw_datasets/inflation_data.csv")

inflation = inflation.drop('amount', axis=1)
inflation = inflation.rename(columns = {'year': 'inflationyear', 'inflation rate': 'inflation'})

#inflation.head()

In [96]:
# 13330 rows, 2 cols
fields = ['date', 'value']
tipo_de_cambio = pd.read_csv("/content/raw_datasets/Tipo_de_cambio.csv", skiprows=15, skipinitialspace=True, usecols=fields)

clean_tipo_de_cambio = tipo_de_cambio.dropna()
clean_tipo_de_cambio = tipo_de_cambio.drop_duplicates()


clean_tipo_de_cambio['date'] = clean_tipo_de_cambio['date'].str[:4].astype(int)

updated_tipo_de_cambio = pd.DataFrame(columns=fields)

t_year = 0
accum = 0.0
count = 0

for idx, row in clean_tipo_de_cambio.iterrows():
  if row['date'] != t_year:

    if(t_year != 0):
      updated_tipo_de_cambio = updated_tipo_de_cambio.append({'date': (t_year), 'value': (float) (accum/count)}, ignore_index=True)

    t_year = row['date']
    accum = row['value']
    count=1
  else:
    count+=1
    accum += row['value']

updated_tipo_de_cambio['date'] = updated_tipo_de_cambio['date'].astype(int)

#updated_tipo_de_cambio.head()

In [97]:
# 252243 rows, 5 cols
fields = ['Maker', 'Genmodel_ID', 'Reg_year', 'Ad_Date', 'Price_USD']
anuncios_usados = pd.read_csv("/content/raw_datasets/Anuncios_usados.csv", skipinitialspace=True, usecols=fields)

# Clean null data
clean_anuncios_usados = anuncios_usados.dropna()
clean_anuncios_usados = anuncios_usados.drop_duplicates()

clean_anuncios_usados = clean_anuncios_usados.dropna(how='any')

clean_anuncios_usados['Reg_year'] = clean_anuncios_usados['Reg_year'].astype(int)

clean_anuncios_usados['Ad_Date'] = clean_anuncios_usados['Ad_Date'].str[-4:].astype(int)

clean_anuncios_usados['Genmodel_ID'] = clean_anuncios_usados['Genmodel_ID'].map(lambda x: x.replace('_',''))

#clean_anuncios_usados.head()

In [98]:
fields = ['Genmodel_ID', 'Reg_year', 'Ad_Date', 'Price_GBP']
updated_precio_historico = pd.DataFrame(columns=fields)
union = pd.merge(clean_anuncios_usados, updated_tipo_de_cambio, left_on='Ad_Date', right_on='date')
union['Price_GBP'] = union['Price_USD'] / union['value'] 
union = union[['Genmodel_ID', 'Reg_year', 'Ad_Date', 'Price_GBP']]

#union.head()

# Output the dataframe to csv if desired

In [83]:
# Models

clean_modelos.to_csv('Modelos.csv', index=False)
files.download('Modelos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
# New sales

clean_ventas_nuevas.to_csv('Ventas_Nuevas.csv', index=False)
files.download('Ventas_Nuevas.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [85]:
# VersionesAvgPrice

clean_df.to_csv('VersionesAvgPrice.csv', index=False)
files.download('VersionesAvgPrice.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [86]:
# Inflation data

inflation.to_csv('inflation_data.csv', index=False)
files.download('inflation_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [87]:
# Anuncios_usadosGBP

union.to_csv('anuncios_usadosgbp.csv', index=False)
files.download('anuncios_usadosgbp.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Join the cleaned datasets into new tables

The result of this new tables will be exported to .csv as well.